# Document Preprocessing

In [1]:
import os
# Setting temp dir to /srv/data directory, as it otherwise fills up the home directory too much
# Just comment out on machines that are not "Goober"
os.environ["TMPDIR"] = "/srv/data/tmp"
os.makedirs("/srv/data/tmp", exist_ok=True)

%pip install haystack-ai
%pip install nltk
%pip install markdown-it-py
%pip install mdit_plain
%pip install openai
%pip install pandas
%pip install hf_xet
%pip install ollama-haystack==2.4.2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pipelines.preprocessing_pipelines.markdown_pipeline import pipeline as markdown_pipeline
from pipelines.preprocessing_pipelines.preprocessing_pipeline import pipeline as preprocessing_pipeline
from haystack.dataclasses import ByteStream
from utils.markdown_utils import for_each_markdown_file
from tqdm import tqdm
import pandas as pd
import uuid

tqdm.pandas()

In [24]:
splitting_configs = [
    {
        "split_by": "word",
        "split_length": 48,
        "split_overlap": 10 # 20%
    },
    {
        "split_by": "word",
        "split_length": 96,
        "split_overlap": 20 # 20%
    },
    {
        "split_by": "word",
        "split_length": 384,
        "split_overlap": 80 # 20%
    },
    {
        "split_by": "word",
        "split_length": 768,
        "split_overlap": 160 # 20%
    },
    {
        "split_by": "line",
        "split_length": 1,
        "split_overlap": 0
    },
    {
        "split_by": "passage",
        "split_length": 1,
        "split_overlap": 0
    },
]

In [25]:
from functools import partial

def preprocess_markdown(filename: str, bytes, splitting_config):
    bytestream = ByteStream(data=bytes)
    file_id = str(uuid.uuid4())
    md_documents = markdown_pipeline.run({
        "markdown_converter" : {
            "byte_streams": [bytestream],
            "id": file_id,
            "meta": {
                "title": filename
            }
        }
    })["markdown_converter"]["documents"]
    print(md_documents)
    splitter = preprocessing_pipeline.get_component("splitter")
    splitter.split_by = splitting_config["split_by"]
    splitter.split_length = splitting_config["split_length"]
    splitter.split_overlap = splitting_config["split_overlap"]
    preprocessed_documents = preprocessing_pipeline.run({
        "splitter": { "documents": md_documents }
    })["filter"]["documents"]
    data.extend([{ "document": document } for document in preprocessed_documents])

for config in splitting_configs:
    data = []
    process_with_config = partial(preprocess_markdown, splitting_config=config)
    for_each_markdown_file("./data", process_with_config)
    df = pd.DataFrame(data)
    df.to_pickle(f"data/preprocessed_documents/docs_{config['split_by']}_{config['split_length']}_{config['split_overlap']}.pkl")


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 533.76it/s]


[Document(id=f3704eb4-2865-4cc9-bc53-eb7480d6be44, content: 'Weshalb wird eine äußere Wendung in Erwägung gezogen?

Im Laufe der Schwangerschaft kann das ungebor...', meta: {'title': 'Äußere Wendung.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 222.56it/s]


[Document(id=b43597ec-6c12-471a-a3de-117c5d753c70, content: 'Ein Kaiserschnitt kann aus unterschiedlichen Gründen und mit variierender Dringlichkeit notwendig we...', meta: {'title': 'Kaiserschnitt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 205.91it/s]


...

[Document(id=f0f0acd1-5406-47b1-a692-32f791e5d5dd, content: 'Die Geburt stellt einen natürlichen Ablauf dar. Durch moderne geburtshilfliche Verfahren, insbesonde...', meta: {'title': 'Medizinische Maßnahmen rund um die Geburt.md'})]


Converting markdown files to Documents: 100%|██████████| 1/1 [00:00<00:00, 165.10it/s]


[Document(id=7f826fcf-8164-4691-adf1-8721a23f0268, content: 'Damit Sie den geplanten Eingriff schmerzfrei erleben, stehen zwei grundlegende Methoden zur Verfügun...', meta: {'title': 'Narkose.md'})]


Processing Markdown files: 100%|██████████| 6/6 [00:00<00:00, 116.56it/s]

[Document(id=21c21278-86cc-4963-88d4-427ea2da45b0, content: 'In bestimmten Situationen wird ein vorzeitiger Geburtsbeginn medizinisch empfohlen oder notwendig:

...', meta: {'title': 'Einleitung der Geburt.md'})]


In [10]:
len(df)

415

In [5]:
document_ids = df["document"].apply(lambda doc: doc.id)
is_unique = document_ids.is_unique

print("All document IDs are unique:", is_unique)


All document IDs are unique: True


In [27]:
# Extract character length of each document
df['content_length'] = df['document'].apply(lambda x: len(x.content))

# Compute statistics
mean_length = df['content_length'].mean()
median_length = df['content_length'].median()
variance_length = df['content_length'].var()
min_length = df['content_length'].min()

print("Mean content length:", mean_length)
print("Median content length:", median_length)
print("Variance of content length:", variance_length)
print("Min content length:", min_length)

Mean content length: 307.30841121495325
Median content length: 285.0
Variance of content length: 33686.670210280376
Min content length: 15


## Splitting Evaluation

Which splitting strategy works best?
- Number of chars (with overlap)
- Number of sentences (with overlap)
- By line
- By paragraph